# Riconoscimento di Animali per Auto a Guida Autonoma

---

### Obiettivo del progetto

Questo notebook presenta un sistema di **riconoscimento automatico di immagini** progettato per distinguere tra **veicoli** e **animali** utilizzando tecniche di Deep Learning.

### Contesto applicativo

Nel contesto delle **auto a guida autonoma**, è fondamentale che il sistema di visione artificiale sia in grado di:
- **Identificare rapidamente** la presenza di animali sulla strada (cani, gatti, cervi, uccelli, ecc.)
- **Distinguerli da veicoli** (automobili, camion)
- **Reagire in modo appropriato** per garantire la sicurezza di tutti

Un errore di classificazione potrebbe avere conseguenze gravi: confondere un animale con un veicolo potrebbe portare a decisioni di guida inadeguate.

### Cosa realizzerò

In questo progetto implementerò:

1. **Preprocessing del dataset CIFAR-10**: selezione e preparazione delle classi rilevanti
2. **Rete Neurale Convoluzionale (CNN)**: architettura custom per la classificazione binaria
3. **Addestramento con callback**: per evitare overfitting e ottimizzare le prestazioni
4. **Metriche di valutazione**: accuracy, precision e matrice di confusione
5. **Analisi delle misclassificazioni**: per comprendere i limiti del modello e possibili miglioramenti

### Metodologia

Il progetto utilizza esclusivamente concetti e tecniche trattate nel corso di Deep Learning:
- Reti Neurali Convoluzionali (CNN)
- Layer di Pooling e Convoluzione
- Funzioni di attivazione (ReLU, Sigmoid)
- Ottimizzazione con Adam
- Tecniche di regolarizzazione (Dropout)
- Analisi degli errori e interpretazione dei risultati

---

**Autore**: Guido Pacciani  
**Progetto**: VisionTech Solutions  
**Master**: AI Engineering - ProfessionAI  
**Dataset**: CIFAR-10 (Canadian Institute For Advanced Research)  
**Framework**: TensorFlow/Keras


## 1. Import delle librerie e configurazione dell'ambiente

In questa sezione importo tutte le librerie necessarie per il progetto e configuro l'ambiente di lavoro per garantire la **riproducibilità** dei risultati.


In [ ]:
# ───────────────────────────────────────────────────────────────
# IMPORT PRINCIPALI
# ───────────────────────────────────────────────────────────────
import os                                   # Operazioni sul sistema operativo (es. variabili d'ambiente)
import random                               # Gestione della casualità e riproducibilità degli esperimenti
import platform                             # Informazioni sul sistema operativo e versione di Python

# Librerie scientifiche di base
import numpy as np                          # Operazioni numeriche e manipolazione di array (es. immagini)
import pandas as pd                         # Dati tabulari (es. metriche e risultati)
import matplotlib.pyplot as plt             # Visualizzazione di immagini e grafici
import matplotlib.patches as mpatches       # Elementi grafici aggiuntivi per legende e annotazioni

# Riduzione del rumore dei log di TensorFlow e forzatura di operazioni deterministiche
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")      # 0=all, 1=info, 2=warning, 3=error
os.environ.setdefault("TF_DETERMINISTIC_OPS", "1")

# TensorFlow e Keras (framework per il deep learning)
try:
    import tensorflow as tf                                 # Libreria principale per il deep learning
    from tensorflow import keras                            # API di alto livello per modelli e layer
    from tensorflow.keras import layers, models             # Layer e modelli Keras
    from tensorflow.keras.datasets import cifar10           # Dataset CIFAR-10 predefinito in Keras
    from tensorflow.keras.callbacks import EarlyStopping    # Callback per l'early stopping durante l'addestramento
    from tensorflow.keras.utils import to_categorical       # Utility per la conversione delle etichette in one-hot encoding
except Exception as e:                                      # Cattura l'errore di importazione nella variabile 'e'
    raise SystemExit(                                       # SystemExit termina l'esecuzione dello script con un messaggio di errore     
        f"TensorFlow/Keras non importabili: {e}\n"
        "Suggerimento: assicurati di essere nel venv corretto e che TF sia installato "
        "(su Apple Silicon: 'tensorflow-macos' + 'tensorflow-metal')."
    )

# ───────────────────────────────────────────────────────────────
# CONFIGURAZIONE DELL'AMBIENTE DI LAVORO
# ───────────────────────────────────────────────────────────────
SEED = 42                                   # Seed costante per garantire risultati replicabili
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Impostazione dello stile grafico
plt.style.use("default")

# ───────────────────────────────────────────────────────────────
# VERIFICA DELL'AMBIENTE
# ───────────────────────────────────────────────────────────────

# Raccolta delle versioni delle librerie principali
python_version = platform.python_version()
tensorflow_version = tf.__version__
keras_version = getattr(keras, '__version__', 'n/d')
numpy_version = np.__version__
pandas_version = pd.__version__

print("=" * 60)
print("🔧 CONFIGURAZIONE AMBIENTE")
print("=" * 60)
print(f"Python version   : {python_version}")
print(f"TensorFlow       : {tensorflow_version}")
print(f"Keras            : {keras_version}")
print(f"NumPy            : {numpy_version}")
print(f"Pandas           : {pandas_version}")
print(f"Seed impostato   : {SEED}")
print("=" * 60)
print("Ambiente configurato correttamente!\n")


: 

## 2. Caricamento e analisi del dataset CIFAR-10

### Informazioni sul dataset

Il **CIFAR-10** è un dataset di riferimento nel campo della Computer Vision, composto da:
- **60.000 immagini a colori** (50.000 training + 10.000 test)
- **Dimensione**: 32x32 pixel, 3 canali RGB
- **10 classi**: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck

### Obiettivo

Per il mio sistema di guida autonoma, riorganizzerò queste 10 classi in **2 macro-categorie**:

1. **ANIMALI** (classe 0): bird, cat, deer, dog, frog, horse
2. **VEICOLI** (classe 1): airplane, automobile, ship, truck

Questo mi permetterà di creare un **classificatore binario** specifico per l'identificazione di ostacoli animati.


In [ ]:
# ───────────────────────────────────────────────────────────────
# CARICAMENTO DEL DATASET CIFAR-10
# ───────────────────────────────────────────────────────────────

# Carico il dataset CIFAR-10 direttamente da Keras
# Questo fornisce automaticamente la divisione train/test
print("Caricamento del dataset CIFAR-10...")
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Definisco i nomi delle classi per una migliore interpretabilità
# Queste corrispondono agli indici 0-9 del dataset originale
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

# ───────────────────────────────────────────────────────────────
# DIMENSIONI DEL DATASET CIFAR-10
# ───────────────────────────────────────────────────────────────

# Ogni immagine è un array 3D: (altezza, larghezza, canali)
# L'intero dataset è un array 4D: (numero_immagini, altezza, larghezza, canali)

# Numero totale di immagini di TRAINING
n_train_images  = x_train.shape[0]   

# Numero totale di immagini di TEST
n_test_images   = x_test.shape[0]    

# Dimensioni delle immagini
img_height      = x_train.shape[1]   # Numero di righe (pixel in altezza)
img_width       = x_train.shape[2]   # Numero di colonne (pixel in larghezza)
img_channels    = x_train.shape[3]   # Numero di canali colore (3 per RGB)

# ───────────────────────────────────────────────────────────────
# STAMPA DELLE INFORMAZIONI PRINCIPALI
# ───────────────────────────────────────────────────────────────
print("Dataset CIFAR-10 caricato correttamente!\n")
print("Dimensioni del dataset:")
print(f"   - Training set : {n_train_images} immagini di dimensione {img_height}×{img_width}")
print(f"   - Test set     : {n_test_images} immagini")
print(f"   - Canali colore: {img_channels} (RGB)")
print(f"   - Range valori pixel: [{x_train.min()}, {x_train.max()}]")

In [ ]:
# ───────────────────────────────────────────────────────────────
# VISUALIZZAZIONE DI CAMPIONI CASUALI DAL DATASET
# ───────────────────────────────────────────────────────────────
# Scopo: visualizzare alcune immagini del dataset per comprendere
#        la varietà, la qualità e la struttura dei dati che useremo
#        per addestrare la rete neurale (CIFAR-10 in questo caso).
# ───────────────────────────────────────────────────────────────

def visualizza_campioni_dataset(x, y, class_names, n_samples=20):
    """
    Visualizza un campione casuale di immagini dal dataset.

    Parametri:
        x (ndarray): array 4D di immagini con forma (n_img, altezza, larghezza, canali) (= 50000, 32, 32, 3)
                     contiene i valori dei pixel (0-255)
        y (ndarray): array 2D di etichette di forma (n_img, 1)
                     contiene i numeri interi delle classi (0–9)
        class_names (list): lista dei nomi delle classi, in ordine (es. ["airplane", "automobile", ...])
        n_samples (int): numero di immagini da visualizzare (default = 20)
    """

    # ───────────────────────────────────────────────
    # 1. Creazione della griglia di sottografici
    # ───────────────────────────────────────────────
    # Crea una figura composta da 4 righe e 5 colonne = 20 riquadri (axes)
    # fig  → l'intera figura ("il foglio")
    # axes → array 4x5 di singoli riquadri (dove verranno disegnate le immagini)
    fig, axes = plt.subplots(4, 5, figsize=(15, 12))

    # Aggiunge un titolo generale alla figura
    fig.suptitle("Campioni dal Dataset CIFAR-10", fontsize=16, fontweight="bold")

    # ───────────────────────────────────────────────
    # 2. Selezione casuale delle immagini
    # ───────────────────────────────────────────────
    # len(x) = numero totale di immagini disponibili
    total_images = len(x)
    
    # Estrae n_samples indici casuali diversi tra loro (replace=False) e li memorizza in 'indices'
    indices = np.random.choice(total_images, n_samples, replace=False)
    
    # Trasforma la matrice 4x5 di riquadri in un vettore lineare da 20 elementi
    # in modo da poter iterare facilmente su tutti i riquadri con un solo ciclo for
    axes = axes.flatten()

    # ───────────────────────────────────────────────
    # 3. Ciclo di visualizzazione delle immagini
    # ───────────────────────────────────────────────
    for posizione, riquadro in enumerate(axes):

        # 1. Prendo l'indice casuale corrispondente a questo riquadro
        indice_immagine = indices[posizione]

        # 2. Estraggo l'immagine e la sua etichetta dal dataset
        immagine = x[indice_immagine]           # array 32x32x3 (RGB)
        etichetta = y[indice_immagine][0]       # es. 3 → 'cat'. Se utilizzassi solo y[i] otterrei un array di 1 elemento
        nome_classe = class_names[etichetta]    # converto l'indice numerico in nome testuale

        # 3. Mostro l'immagine nel riquadro corrispondente
        riquadro.imshow(immagine)                       # visualizza l'immagine (valori 0–255)
        riquadro.set_title(nome_classe, fontsize=10)    # scrive sopra il nome della classe
        riquadro.axis('off')                            # rimuove assi e numeri per rendere il layout pulito

    # ───────────────────────────────────────────────
    # 4. Ottimizzazione del layout e visualizzazione
    # ───────────────────────────────────────────────
    plt.tight_layout()              # Riduce gli spazi vuoti tra i riquadri
    plt.subplots_adjust(top=0.9)    # Sposta il titolo generale verso l'alto
    plt.show()                      # Mostra la figura finale

# Visualizzo i campioni
visualizza_campioni_dataset(x_train, y_train, class_names) 


### Conversione in classificazione binaria

Ora converto le 10 classi originali in 2 macro-categorie per il mio task specifico.


In [ ]:
# ───────────────────────────────────────────────────────────────
# CONVERSIONE CIFAR-10 → CLASSIFICAZIONE BINARIA (ANIMALI vs VEICOLI)
# ───────────────────────────────────────────────────────────────
# Obiettivo: mappare le 10 classi originali di CIFAR-10 in 2 classi:
#            0 = ANIMALE (bird, cat, deer, dog, frog, horse)
#            1 = VEICOLO (airplane, automobile, ship, truck)
# ───────────────────────────────────────────────────────────────

# Indici originali CIFAR-10 (promemoria):
# 0: airplane | 1: automobile | 2: bird | 3: cat | 4: deer  
# 5: dog      | 6: frog       | 7: horse| 8: ship| 9: truck

ANIMALI_IDX = np.array([2, 3, 4, 5, 6, 7], dtype=int)   # bird, cat, deer, dog, frog, horse
VEICOLI_IDX = np.array([0, 1, 8, 9], dtype=int)         # airplane, automobile, ship, truck

def converti_a_binario(y):
    """
    Converte le etichette CIFAR-10 (0–9) in etichette binarie:
        0 = animale, 1 = veicolo.

    Parametri:
        y (ndarray): array delle etichette originali, forma (n, 1) oppure (n,)

    Ritorna:
        y_bin (ndarray): array 1D di forma (n,) con valori {0, 1}
    """
    # 1. Garanzia di forma 1D (es. (50000,) anziché (50000,1))
    y_1d = np.squeeze(y)        # rimuove la seconda dimensione (inutile) e appiattisce l’array: (n,1) → (n,)
    y_1d = y_1d.astype(int)     # converte tutti i valori in interi (necessario per usarli come indici o etichette) 

    # 2. Alloco l'array binario (stesso numero di campioni)
    y_bin = np.zeros_like(y_1d, dtype=np.int32) # crea un nuovo array di zeri con la stessa forma di y_1d

    # 3. Mappatura vettoriale: assegno 1 ai veicoli e lascio 0 agli animali
    #
    # np.isin(y_1d, VEICOLI_IDX) restituisce un array di valori booleani (True/False)
    # di lunghezza pari a y_1d: True dove l'etichetta in y_1d appartiene a VEICOLI_IDX, False altrove.
    #
    # Esempio:
    #   y_1d        = [2, 0, 8, 4, 7]
    #   VEICOLI_IDX = [0, 1, 8, 9]
    #   np.isin(y_1d, VEICOLI_IDX) → [False, True, True, False, False]
    #
    # Usando questa "maschera booleana" come indice di y_bin,
    # NumPy seleziona automaticamente tutte le posizioni True
    # e assegna il valore 1 a quelle celle in un'unica operazione vettoriale.
    #
    # Questa strategia è molto più efficiente di un ciclo for,
    # poiché NumPy esegue l'operazione a livello di codice C, in parallelo,
    # evitando l'iterazione elemento per elemento in Python.
    y_bin[np.isin(y_1d, VEICOLI_IDX)] = 1

    return y_bin

def stampa_distribuzione_binaria(y_bin, titolo="Distribuzione classi (binario)"):
    """
    Stampa conteggi e percentuali delle classi binarie {0,1}.
    """
    n = y_bin.shape[0]                      # shape[0] ritorna il numero di oggetti
    n_animali = int(np.sum(y_bin == 0))     # ritorna il numero totale di 0
    n_veicoli = int(np.sum(y_bin == 1))     # ritorna il numero totale di 1
    p_animali = 100.0 * n_animali / n       # percentuale di animali
    p_veicoli = 100.0 * n_veicoli / n       # percentuale di veicoli

    print(titolo)
    print(f"   - Animali (0): {n_animali} immagini ({p_animali:.1f}%)")
    print(f"   - Veicoli (1): {n_veicoli} immagini ({p_veicoli:.1f}%)")
    print()

# ───────────────────────────────────────────────────────────────
# APPLICAZIONE DELLA CONVERSIONE + VERIFICA DISTRIBUZIONE
# ───────────────────────────────────────────────────────────────
y_train_binary = converti_a_binario(y_train)
y_test_binary  = converti_a_binario(y_test)

stampa_distribuzione_binaria(y_train_binary, titolo="Training set — distribuzione binaria")
stampa_distribuzione_binaria(y_test_binary,  titolo="Test set — distribuzione binaria")

# Nota: Un dataset ~bilanciato (≈50/50) aiuta a evitare bias verso una classe.


### Normalizzazione delle immagini

La normalizzazione è un passo fondamentale nel preprocessing: porta i valori dei pixel da [0, 255] a [0, 1], facilitando la convergenza durante l'addestramento.


In [ ]:
# ───────────────────────────────────────────────────────────────
# NORMALIZZAZIONE DELLE IMMAGINI E PREPARAZIONE DEI DATI
# ───────────────────────────────────────────────────────────────
# Scopo: portare i valori dei pixel da [0, 255] a [0, 1]
#        per migliorare la stabilità numerica durante l’addestramento
#        (le reti neurali convergono più facilmente con input normalizzati).
# ───────────────────────────────────────────────────────────────

# 1. Conversione del tipo di dato e normalizzazione
# Ogni immagine (32×32×3) contiene valori di pixel interi [0,255].
# Convertiamo in float32 e dividiamo per 255 → range [0,1].
x_train_norm = x_train.astype(np.float32) / 255.0
x_test_norm  = x_test.astype(np.float32)  / 255.0

# 2. Verifica dei range dopo la normalizzazione
x_train_min, x_train_max = x_train_norm.min(), x_train_norm.max()
x_test_min,  x_test_max  = x_test_norm.min(),  x_test_norm.max()

print("Normalizzazione completata!\n")
print("Range dei valori dopo normalizzazione:")
print(f"   - Training set: [{x_train_min:.2f}, {x_train_max:.2f}]")
print(f"   - Test set:     [{x_test_min:.2f}, {x_test_max:.2f}]")

# 3. Conversione delle etichette in float32
# Le etichette binarie (0=animale, 1=veicolo) vengono convertite in float32
# per compatibilità con i tensori di TensorFlow/Keras (richiesti in formato float).
y_train_binary = y_train_binary.astype(np.float32)
y_test_binary  = y_test_binary.astype(np.float32)

# 4. Riepilogo finale dei dataset pronti per l’addestramento

print("\n Riepilogo dei dataset preparati per il modello\n")

print(f" - X_train: array contenente gli input di addestramento "
      f"→ shape = {x_train_norm.shape} "
      f"(n_immagini, altezza, larghezza, canali)")

print(f" - y_train: array contenente le etichette di addestramento "
      f"→ shape = {y_train_binary.shape} "
      f"(n_etichette, ) corrispondenti a ciascun elemento di X_train")

print(f" - X_test : array contenente gli input di valutazione "
      f"→ shape = {x_test_norm.shape} "
      f"(n_immagini_test, altezza, larghezza, canali)")

print(f" - y_test : array contenente le etichette di valutazione "
      f"→ shape = {y_test_binary.shape} "
      f"(n_etichette_test, ) corrispondenti a ciascun elemento di X_test")

## 3. Definizione della Rete Neurale Convoluzionale (CNN)

### Architettura del modello

Costruirò una **CNN custom** per la classificazione binaria. L'architettura si basa sui principi appresi nel corso:

#### Componenti principali:

1. **Layer Convoluzionali (Conv2D)**: estraggono caratteristiche locali dalle immagini (bordi, forme, texture)
2. **Layer di Pooling (MaxPooling2D)**: riducono la dimensionalità mantenendo le informazioni più importanti
3. **Dropout**: tecnica di regolarizzazione per prevenire l'overfitting
4. **Layer Densi (Dense)**: combinano le caratteristiche estratte per la classificazione finale

### Scelte architetturali

- **ReLU** come funzione di attivazione nascosta (evita il problema del gradiente nullo)
- **Sigmoid** per l'output (classificazione binaria → probabilità tra 0 e 1)
- **Pooling** per ridurre i parametri e aumentare l'invarianza traslazionale
- **Dropout** (40%) per regolarizzazione


In [ ]:
# ───────────────────────────────────────────────────────────────
# ARCHITETTURA CNN (CIFAR-10 → BINARIO: ANIMALE vs VEICOLO)
# ───────────────────────────────────────────────────────────────
# Input: immagine in formato matriciale 32×32×3 (RGB)
# 
#  ┌──────────────────────────────────────────────────────────┐
#  │ BLOCCO CONV 1                                            │
#  │   Conv2D(32, 3×3, padding='same', ReLU)                  │  → 32×32×32
#  │   BatchNorm                                              │  → 32×32×32
#  │   MaxPool(2×2)                                           │  → 16×16×32
#  └──────────────────────────────────────────────────────────┘
#
#  ┌──────────────────────────────────────────────────────────┐
#  │ BLOCCO CONV 2                                            │
#  │   Conv2D(64, 3×3, padding='same', ReLU)                  │  → 16×16×64
#  │   BatchNorm                                              │  → 16×16×64
#  │   MaxPool(2×2)                                           │  →  8×8×64
#  └──────────────────────────────────────────────────────────┘
#
#  ┌──────────────────────────────────────────────────────────┐
#  │ BLOCCO CONV 3                                            │
#  │   Conv2D(128, 3×3, padding='same', ReLU)                 │  →  8×8×128
#  │   BatchNorm                                              │  →  8×8×128
#  │   MaxPool(2×2)                                           │  →  4×4×128
#  └──────────────────────────────────────────────────────────┘
#
#  Flatten                                                    │  →  2048
#  Dense(128, ReLU)                                           │
#  Dropout(0.4)                                               │
#  Dense(64,  ReLU)                                           │
#  Dropout(0.3)                                               │
#  Dense(1,  Sigmoid)  → p(veicolo) ∈ [0,1]                   │
# ───────────────────────────────────────────────────────────────
# NOTE:
# - Dopo i 3 MaxPool: 32→16→8→4 (dimensione spaziale).
# - 4×4×128 = 2048 feature prima dei densi.
# - Output binario con Sigmoid: <0.5=animale (0), ≥0.5=veicolo (1).
# ───────────────────────────────────────────────────────────────


# Definisco una funzione per creare il modello
def crea_modello_cnn(input_shape=(32, 32, 3)):
    """
    Crea e restituisce un modello CNN per classificazione binaria
    (0=animale, 1=veicolo) su immagini 32×32×3.

    Args:
        input_shape (tuple): (altezza, larghezza, canali), default (32, 32, 3).

    Returns:
        model (keras.Model): modello Keras NON compilato.
    """
    # Inizializzatore di He (HeNormal):
    #   - Distribuisce i pesi iniziali secondo una gaussiana N(0, sqrt(2/n_in))
    #   - Ottimizzato per layer con attivazione ReLU
    #   - Mantiene la varianza dei segnali costante tra i layer,
    #     evitando esplosioni o annullamenti del gradiente durante il training
    he = keras.initializers.HeNormal()
    
    model = models.Sequential(name="CNN_animali_vs_veicoli")

    # ───────────────── BLOCCO CONV 1 ─────────────────
    model.add(layers.Conv2D(
        32, (3, 3), activation='relu', padding='same',
        kernel_initializer=he, name='conv1', input_shape=input_shape
    ))
    model.add(layers.BatchNormalization(name='bn1'))
    model.add(layers.MaxPooling2D((2, 2), name='pool1'))  # 32→16

    # ───────────────── BLOCCO CONV 2 ─────────────────
    model.add(layers.Conv2D(
        64, (3, 3), activation='relu', padding='same',
        kernel_initializer=he, name='conv2'
    ))
    model.add(layers.BatchNormalization(name='bn2'))
    model.add(layers.MaxPooling2D((2, 2), name='pool2'))  # 16→8

    # ───────────────── BLOCCO CONV 3 ─────────────────
    model.add(layers.Conv2D(
        128, (3, 3), activation='relu', padding='same',
        kernel_initializer=he, name='conv3'
    ))
    model.add(layers.BatchNormalization(name='bn3'))
    model.add(layers.MaxPooling2D((2, 2), name='pool3'))  # 8→4

    # ──────────────── TESTA DENSE ────────────────────
    model.add(layers.Flatten(name='flatten'))                 # 4×4×128=2048
    model.add(layers.Dense(128, activation='relu', kernel_initializer=he, name='dense1'))
    model.add(layers.Dropout(0.4, name='dropout1'))
    model.add(layers.Dense(64, activation='relu', kernel_initializer=he, name='dense2'))
    model.add(layers.Dropout(0.3, name='dropout2'))

    # ───────────────── OUTPUT ────────────────────────
    model.add(layers.Dense(1, activation='sigmoid', name='output'))

    return model


# Funzione di compilazione (separata)
def compila_modello(model, lr=1e-3):
    """
    Compila il modello per classificazione binaria.

    Args:
        model (keras.Model): modello da compilare.
        lr (float): learning rate dell’ottimizzatore (default=0.001)

    Returns:
        model (keras.Model): modello compilato.
    """
    optim = keras.optimizers.Adam(learning_rate=lr)
    model.compile(
        optimizer=optim,
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.AUC(name='auc')
        ]
    )
    return model


# ───────────────────────────────────────────────────
# Costruzione + riepilogo
# ───────────────────────────────────────────────────
print("Costruzione del modello CNN...\n")
model = crea_modello_cnn()
model = compila_modello(model)

# model.summary() mostra l'architettura completa con il numero di parametri
model.summary(line_length=110, expand_nested=False)

print("\n" + "="*60)
print("Modello creato e compilato con successo!")
print("="*60)

## 4. Compilazione e addestramento del modello

### Configurazione dell'addestramento

Prima di addestrare il modello, dobbiamo configurare:

1. **Loss Function**: `binary_crossentropy` - ideale per classificazione binaria
2. **Optimizer**: `Adam` - algoritmo di ottimizzazione adattivo che combina i vantaggi di RMSprop e momentum
3. **Metriche**: `accuracy` - percentuale di predizioni corrette
4. **Callbacks**: `EarlyStopping` - interrompe l'addestramento se la validation loss non migliora, prevenendo overfitting


In [ ]:
# ───────────────────────────────────────────────────────────────
# COMPILAZIONE DEL MODELLO: LOSS, OTTIMIZZATORE, METRICHE
# ───────────────────────────────────────────────────────────────
# Scopo: definire la funzione di costo, l’algoritmo di ottimizzazione e
#        le metriche di valutazione per una classificazione binaria
#        con output sigmoide (p ∈ [0,1]).
# ───────────────────────────────────────────────────────────────

# 1. Loss function (funzione di costo)
# - "binary_crossentropy" misura la distanza tra etichetta vera (0/1)
#   e probabilità predetta (output sigmoide).
# - Da usare quando l’ultimo layer è Dense(1, activation='sigmoid').
loss_function = 'binary_crossentropy'

# 2. Ottimizzatore
# - Adam: ottimizzatore adattivo che unisce momentum e adattamento del learning rate
#   per parametro. È una scelta robusta di default.
# - learning_rate=1e-3 è un punto di partenza tipico e stabile.
optimizer = keras.optimizers.Adam(learning_rate=1e-3)

# 3. Metriche di valutazione
#   - accuracy  → percentuale di predizioni corrette.
#   - precision → misura la purezza delle predizioni positive.
#                 (quanto spesso, quando predico "1", ho davvero ragione?)
#                 Formula: TP / (TP + FP)
#   - recall    → sensibilità. Misura la capacità di catturare tutti i positivi reali.
#                 (quante volte, tra tutti i veri "1", li ho riconosciuti come tali?)
#                 Formula: TP / (TP + FN)
#   - AUC       → area sotto la curva ROC.
#                 Misura la capacità del modello di distinguere tra classi,
#                 indipendentemente dalla soglia di decisione.
#                 0.5 = casuale, 1.0 = perfetto.
metrics=[
    'accuracy',
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc')
]


# 4. Compilazione effettiva del modello
model.compile(
    optimizer=optimizer,
    loss=loss_function,
    metrics=metrics
)

# 5. Riepilogo configurazione
print("Modello compilato con successo!\n")
print("Configurazione di training:")
print(f"   • Loss function : {loss_function}")
print(f"   • Optimizer     : Adam (learning_rate={optimizer.learning_rate.numpy():.3f})")
print("    • Metriche      :")
print("        - accuracy : percentuale di predizioni corrette")
print("        - precision: qualità delle predizioni positive (TP / (TP+FP))")
print("        - recall   : capacità di trovare tutti i positivi (TP / (TP+FN))")
print("        - AUC      : capacità discriminante complessiva (ROC area)\n")


In [ ]:
# ───────────────────────────────────────────────────────────────
# ADDESTRAMENTO DEL MODELLO CON EARLY STOPPING
# ───────────────────────────────────────────────────────────────
# Scopo: addestrare la CNN in modo efficiente, evitando l’overfitting.
#        Se la prestazione sul validation set smette di migliorare
#        per un certo numero di epoche consecutive, l’addestramento
#        viene interrotto automaticamente e vengono ripristinati
#        i pesi migliori osservati durante il training.
# ───────────────────────────────────────────────────────────────

# 1. Callback EarlyStopping
# - monitor: indica quale metrica osservare (qui 'val_loss')
# - patience: numero di epoche consecutive senza miglioramento dopo cui interrompere
# - restore_best_weights: ripristina i pesi del modello al punto con val_loss minima
# - verbose=1: mostra un messaggio quando si verifica l’arresto anticipato
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# 2. Parametri principali di training
# - EPOCHS: numero massimo di epoche di addestramento.
#            (il training può fermarsi prima grazie all’early stopping)
# - BATCH_SIZE: numero di campioni elaborati simultaneamente.
#               Valori più piccoli → aggiornamenti più frequenti ma più rumorosi;
#               valori grandi → maggiore stabilità ma più memoria richiesta.
# - VALIDATION_SPLIT: percentuale del training set usata per valutare le prestazioni
#                     su dati mai visti durante l’addestramento.
EPOCHS = 30
BATCH_SIZE = 64
VALIDATION_SPLIT = 0.2

# 3. Riepilogo configurazione
print("Inizio addestramento del modello...\n")
print("Parametri di training:")
print(f"   • Epoche massime          : {EPOCHS}")
print(f"   • Dimensione batch        : {BATCH_SIZE}")
print(f"   • Validation split        : {VALIDATION_SPLIT * 100:.0f}%")
print(f"   • Early stopping patience : {early_stopping.patience} epoche")
print("\n" + "=" * 60 + "\n")

# 4. Avvio dell’addestramento
# - model.fit() esegue il training vero e proprio.
# - validation_split suddivide automaticamente una parte del training set
#   per valutare le prestazioni del modello a ogni epoca.
# - callbacks=[early_stopping] abilita il monitoraggio continuo delle metriche.
history = model.fit(
    x_train_norm,
    y_train_binary,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=[early_stopping],
    verbose=1
)

# 5. Fine training
print("\n" + "=" * 60)
print("Addestramento completato!")
print("=" * 60)


### Visualizzazione dell'andamento del Training

Analizzo l'andamento di loss e accuracy durante l'addestramento per verificare che il modello stia apprendendo correttamente e non sia in overfitting.


In [ ]:
# ───────────────────────────────────────────────────────────────
# VISUALIZZAZIONE DELL’ANDAMENTO DEL TRAINING
# ───────────────────────────────────────────────────────────────
# Scopo: visualizzare in modo chiaro le curve di training e validation
#        (loss e accuracy) e individuare l’epoca in cui il modello
#        ha raggiunto le prestazioni ottimali (val_loss minima).
# ───────────────────────────────────────────────────────────────

def plot_training_history(history):
    """
    Visualizza l’andamento di loss e accuracy durante il training
    e mostra le metriche relative alla miglior epoca (val_loss minima).

    Args:
        history (keras.callbacks.History): oggetto History restituito da model.fit()
    """
    # ───────────────────────────────────────────────
    # 1. Estrazione dei dati dalla history
    # ───────────────────────────────────────────────
    train_loss = history.history['loss']
    val_loss   = history.history['val_loss']
    train_acc  = history.history['accuracy']
    val_acc    = history.history['val_accuracy']
    epochs     = range(1, len(train_loss) + 1)

    # ───────────────────────────────────────────────
    # 2. Identificazione dell’epoca migliore
    # ───────────────────────────────────────────────
    best_epoch = np.argmin(val_loss) + 1            # +1 perché le epoche partono da 1
    best_val_loss = val_loss[best_epoch - 1]
    best_val_acc  = val_acc[best_epoch - 1]
    best_train_loss = train_loss[best_epoch - 1]
    best_train_acc  = train_acc[best_epoch - 1]

    # ───────────────────────────────────────────────
    # 3. Creazione dei grafici
    # ───────────────────────────────────────────────
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # --- LOSS ---
    ax1.plot(epochs, train_loss, 'b-o', label='Training Loss', linewidth=2)
    ax1.plot(epochs, val_loss, 'r-s', label='Validation Loss', linewidth=2)
    ax1.axvline(best_epoch, color='green', linestyle='--', alpha=0.7, label=f'Best Epoch ({best_epoch})')
    ax1.set_xlabel('Epoca', fontsize=12)
    ax1.set_ylabel('Loss (Binary Crossentropy)', fontsize=12)
    ax1.set_title('Andamento della Loss Function', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)

    # --- ACCURACY ---
    ax2.plot(epochs, train_acc, 'b-o', label='Training Accuracy', linewidth=2)
    ax2.plot(epochs, val_acc, 'r-s', label='Validation Accuracy', linewidth=2)
    ax2.axvline(best_epoch, color='green', linestyle='--', alpha=0.7, label=f'Best Epoch ({best_epoch})')
    ax2.set_xlabel('Epoca', fontsize=12)
    ax2.set_ylabel('Accuracy', fontsize=12)
    ax2.set_title('Andamento dell’Accuracy', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # ───────────────────────────────────────────────
    # 4. Interpretazione e riepilogo della miglior epoca
    # ───────────────────────────────────────────────
    print("\nINTERPRETAZIONE DEI GRAFICI:")
    print("✓ Se le curve di training e validation restano vicine → buona generalizzazione")
    print("✗ Se la validation loss risale mentre la training loss scende → overfitting")
    print("✓ La linea verde indica l’epoca con la minima validation loss.\n")

    print(f"Miglior epoca trovata: {best_epoch}")
    print(f"   • Training Loss : {best_train_loss:.4f}")
    print(f"   • Validation Loss : {best_val_loss:.4f}")
    print(f"   • Training Accuracy : {best_train_acc:.4f} ({best_train_acc*100:.2f}%)")
    print(f"   • Validation Accuracy : {best_val_acc:.4f} ({best_val_acc*100:.2f}%)")

# ───────────────────────────────────────────────────────────────
# ESECUZIONE
# ───────────────────────────────────────────────────────────────
plot_training_history(history)

## 5. Valutazione del modello sul Test Set

Il test set contiene immagini che il modello **non ha mai visto** durante l'addestramento. Questo mi permette di valutare la vera capacità di **generalizzazione** del modello.

### Metriche principali

1. **Accuracy**: Percentuale di predizioni corrette sul totale
   - Formula: (TP + TN) / (TP + TN + FP + FN)
   
2. **Precision**: Tra tutte le predizioni positive (veicoli), quante sono corrette?
   - Formula: TP / (TP + FP)
   - Importante per evitare falsi allarmi

3. **Recall**: Tra tutti i veri positivi (veicoli reali), quanti ne identifichiamo?
   - Formula: TP / (TP + FN)
   - Importante per non perdere casi critici

Dove: TP=True Positive, TN=True Negative, FP=False Positive, FN=False Negative


In [ ]:
# ───────────────────────────────────────────────────────────────
# VALUTAZIONE DEL MODELLO SUL TEST SET
# ───────────────────────────────────────────────────────────────
# Scopo: calcolare le metriche di performance (loss, accuracy,
#        precision, recall, AUC) sul test set, utilizzando i pesi
#        migliori trovati durante l’addestramento (grazie a EarlyStopping).
#        Inoltre, generare le predizioni binarie (0/1) per analisi successive.
# ───────────────────────────────────────────────────────────────

print("Valutazione del modello sul test set...\n")

# 1. Valutazione del modello
# model.evaluate() restituisce un dizionario con le metriche
# specificate nella fase di compilazione.
# Le chiavi attese sono: ['loss', 'accuracy', 'precision', 'recall', 'auc']
eval_res = model.evaluate(x_test_norm, y_test_binary, verbose=0, return_dict=True)

# 2. Predizioni sul test set
# predict() restituisce, per ogni immagine, la probabilità di appartenere
# alla classe "1" (veicolo) nel range [0,1].
y_pred_probs = model.predict(x_test_norm, verbose=0)

# 3. Conversione delle probabilità in classi binarie
# Applichiamo una soglia decisionale fissa:
#   - Probabilità ≥ 0.5 → classe 1 (veicolo)
#   - Probabilità <  0.5 → classe 0 (animale)
y_pred = (y_pred_probs >= 0.5).astype(int)

# 4. Stampa ordinata delle metriche calcolate
print("=" * 60)
print("RISULTATI SUL TEST SET")
print("=" * 60)
print(f"   • Loss       : {eval_res['loss']:.4f}")
print(f"   • Accuracy   : {eval_res['accuracy']:.4f}  ({eval_res['accuracy']*100:.2f}%)")
print(f"   • Precision  : {eval_res['precision']:.4f}  ({eval_res['precision']*100:.2f}%)")
print(f"   • Recall     : {eval_res['recall']:.4f}  ({eval_res['recall']*100:.2f}%)")
print(f"   • AUC        : {eval_res['auc']:.4f}  ({eval_res['auc']*100:.2f}%)")

# 5. Interpretazione sintetica dei risultati
print("\nInterpretazione:")
print(f"   Il modello riconosce correttamente circa il {eval_res['accuracy']*100:.2f}% "
      f"delle immagini del test set.")
print(f"   - Precision  misura la qualità delle predizioni positive "
      f"(quanto spesso, quando predico 'veicolo', ho ragione).")
print(f"   - Recall     misura la capacità di individuare tutti i veicoli reali.")
print(f"   - AUC        ({eval_res['auc']*100:.2f}%) indica l’abilità del modello "
      f"di distinguere tra le due classi a diverse soglie di decisione.")
print(f"   La perdita media (loss = {eval_res['loss']:.4f}) rappresenta "
      f"la distanza media tra le probabilità predette e le etichette reali.")

In [ ]:
# ───────────────────────────────────────────────────────────────
# CALCOLO MANUALE DELLE METRICHE BINARIE
# ───────────────────────────────────────────────────────────────
# Scopo: comprendere in modo trasparente come vengono calcolate
#        le metriche di classificazione (accuracy, precision,
#        recall, F1-score) a partire dalla matrice di confusione.
#        Usiamo solo NumPy, senza librerie esterne.
# ───────────────────────────────────────────────────────────────

def calcola_metriche_binarie(y_true, y_pred):
    """
    Calcola manualmente le metriche di classificazione binaria:
    accuracy, precision, recall, F1-score e componenti della
    matrice di confusione (TP, TN, FP, FN).

    Parametri:
        y_true (ndarray): etichette reali (0=animale, 1=veicolo)
        y_pred (ndarray): etichette predette (0=animale, 1=veicolo)
    
    Ritorna:
        dict contenente TP, TN, FP, FN e le metriche calcolate
    """

    # ───────────────────────────────────────────────
    # 1. Pre-elaborazione
    # ───────────────────────────────────────────────
    # Convertiamo le etichette in array monodimensionali
    # per poter applicare facilmente gli operatori logici.
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    # ───────────────────────────────────────────────
    # 2. Calcolo della matrice di confusione
    # ───────────────────────────────────────────────
    # TP: veicoli predetti correttamente
    TP = np.sum((y_pred == 1) & (y_true == 1))
    # TN: animali predetti correttamente
    TN = np.sum((y_pred == 0) & (y_true == 0))
    # FP: animali scambiati per veicoli
    FP = np.sum((y_pred == 1) & (y_true == 0))
    # FN: veicoli scambiati per animali
    FN = np.sum((y_pred == 0) & (y_true == 1))

    # ───────────────────────────────────────────────
    # 3. Calcolo delle metriche fondamentali
    # ───────────────────────────────────────────────
    # Precision: TP / (TP + FP)
    # → "Di tutte le predizioni 'veicolo', quante sono corrette?"
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0

    # Recall: TP / (TP + FN)
    # → "Di tutti i veri veicoli, quanti ne ho identificati?"
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    # F1-score: media armonica di precision e recall
    # → utile quando il dataset è sbilanciato
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Accuracy: (TP + TN) / (totale)
    # → "Percentuale complessiva di predizioni corrette"
    accuracy = (TP + TN) / (TP + TN + FP + FN)

    # ───────────────────────────────────────────────
    # 4. Restituzione dei risultati in un dizionario
    # ───────────────────────────────────────────────
    return {
        'TP': TP, 'TN': TN, 'FP': FP, 'FN': FN,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    }

# ───────────────────────────────────────────────────────────────
# CALCOLO E STAMPA DEI RISULTATI
# ───────────────────────────────────────────────────────────────
metriche = calcola_metriche_binarie(y_test_binary, y_pred)

# Stampa dettagliata dei risultati
print("\n" + "="*60)
print("METRICHE DETTAGLIATE SUL TEST SET")
print("="*60)

# Matrice di confusione
print(f"\nMatrice di Confusione:")
print(f"   • True Positive (TP):  {metriche['TP']:5d} → veicoli identificati correttamente")
print(f"   • True Negative (TN):  {metriche['TN']:5d} → animali identificati correttamente")
print(f"   • False Positive (FP): {metriche['FP']:5d} → animali scambiati per veicoli")
print(f"   • False Negative (FN): {metriche['FN']:5d} → veicoli scambiati per animali")

# Metriche di performance
print(f"\nMetriche di Performance:")
print(f"   • Accuracy :  {metriche['accuracy']:.4f} ({metriche['accuracy']*100:.2f}%)")
print(f"   • Precision:  {metriche['precision']:.4f} ({metriche['precision']*100:.2f}%)")
print(f"   • Recall   :  {metriche['recall']:.4f} ({metriche['recall']*100:.2f}%)")
print(f"   • F1-Score :  {metriche['f1_score']:.4f}")

# Interpretazione finale
print(f"\nINTERPRETAZIONE:")
print(f"   • Precision ({metriche['precision']*100:.1f}%) → quando il modello predice 'veicolo',")
print(f"     ha ragione nel {metriche['precision']*100:.1f}% dei casi.")
print(f"   • Recall ({metriche['recall']*100:.1f}%) → il modello riconosce correttamente")
print(f"     il {metriche['recall']*100:.1f}% di tutti i veicoli reali.")
print(f"   • F1-Score ({metriche['f1_score']:.3f}) → equilibrio tra precision e recall,")
print(f"     utile per valutare la robustezza generale della classificazione.")

In [ ]:
# ───────────────────────────────────────────────────────────────
# VISUALIZZAZIONE DELLA MATRICE DI CONFUSIONE
# ───────────────────────────────────────────────────────────────
# Scopo: rappresentare graficamente la matrice di confusione
#        per interpretare visivamente le prestazioni del modello.
#        Ogni cella mostra il numero (e la percentuale) di esempi
#        correttamente o erroneamente classificati.
# ───────────────────────────────────────────────────────────────

def plot_confusion_matrix(metriche):
    """
    Visualizza la matrice di confusione (2×2) in formato grafico.

    Parametri:
        metriche (dict): dizionario contenente TP, TN, FP, FN
                         calcolati manualmente.
    """

    # ───────────────────────────────────────────────
    # 1. Costruzione della matrice 2×2
    # ───────────────────────────────────────────────
    # Le righe rappresentano le classi vere,
    # le colonne rappresentano le classi predette.
    # Riga 0 → Animale (0)
    # Riga 1 → Veicolo (1)
    confusion_matrix = np.array([
        [metriche['TN'], metriche['FP']],  # Veri animali
        [metriche['FN'], metriche['TP']]   # Veri veicoli
    ])

    # ───────────────────────────────────────────────
    # 2. Creazione del grafico
    # ───────────────────────────────────────────────
    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(confusion_matrix, cmap='Blues', alpha=0.85)

    # Etichette degli assi
    class_labels = ['Animale (0)', 'Veicolo (1)']
    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(class_labels, fontsize=11)
    ax.set_yticklabels(class_labels, fontsize=11)

    # Titoli e descrizioni
    ax.set_xlabel('Classe Predetta', fontsize=13, fontweight='bold')
    ax.set_ylabel('Classe Vera', fontsize=13, fontweight='bold')
    ax.set_title('Matrice di Confusione', fontsize=15, fontweight='bold', pad=20)

    # ───────────────────────────────────────────────
    # 3. Annotazione dei valori numerici in ogni cella
    # ───────────────────────────────────────────────
    total = len(y_test_binary)
    for i in range(2):
        for j in range(2):
            value = confusion_matrix[i, j]
            percentage = (value / total) * 100
            ax.text(j, i,
                    f"{value}\n({percentage:.1f}%)",
                    ha="center", va="center",
                    fontsize=13, fontweight="bold", color="black")

    # ───────────────────────────────────────────────
    # 4. Legenda interpretativa
    # ───────────────────────────────────────────────
    legend_text = (
        "TN (True Negative): Animali classificati correttamente\n"
        "FP (False Positive): Animali scambiati per veicoli\n"
        "FN (False Negative): Veicoli scambiati per animali\n"
        "TP (True Positive): Veicoli classificati correttamente"
    )

    plt.figtext(0.5, -0.05, legend_text, ha='center', fontsize=10,
                bbox=dict(boxstyle='round,pad=0.5',
                          facecolor='wheat', alpha=0.3))

    plt.tight_layout()
    plt.show()

# ───────────────────────────────────────────────────────────────
# ESECUZIONE DELLA VISUALIZZAZIONE
# ───────────────────────────────────────────────────────────────
plot_confusion_matrix(metriche)


## 6. Analisi delle misclassificazioni

L'analisi qualitativa degli errori è fondamentale per:
- Capire **quali immagini** il modello fatica a classificare
- Identificare **pattern comuni** negli errori
- Suggerire **miglioramenti futuri** all'architettura o al preprocessing

Visualizzerò esempi di:
1. **False Positive (FP)**: Animali classificati come veicoli
2. **False Negative (FN)**: Veicoli classificati come animali


In [ ]:
# ───────────────────────────────────────────────────────────────
# ANALISI DEGLI ERRORI DI CLASSIFICAZIONE
# ───────────────────────────────────────────────────────────────
# Scopo: identificare le immagini che il modello ha classificato
#        in modo errato — sia come *False Positive* (animali scambiati
#        per veicoli) che *False Negative* (veicoli scambiati per animali).
#        Questi indici saranno utili per una successiva analisi visiva.
# ───────────────────────────────────────────────────────────────

def trova_errori(y_true, y_pred):
    """
    Trova gli indici dei campioni classificati in modo errato.

    Parametri:
        y_true (ndarray): etichette reali (0=animale, 1=veicolo)
        y_pred (ndarray): etichette predette (0=animale, 1=veicolo)

    Ritorna:
        tuple: (false_positives, false_negatives)
               - false_positives → indici degli animali scambiati per veicoli
               - false_negatives → indici dei veicoli scambiati per animali
    """

    # ───────────────────────────────────────────────
    # 1. Conversione in array monodimensionali
    # ───────────────────────────────────────────────
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    # ───────────────────────────────────────────────
    # 2. Identificazione degli errori
    # ───────────────────────────────────────────────
    # False Positive → (predetto 1 ma vero 0)
    false_positives = np.where((y_pred == 1) & (y_true == 0))[0]

    # False Negative → (predetto 0 ma vero 1)
    false_negatives = np.where((y_pred == 0) & (y_true == 1))[0]

    return false_positives, false_negatives


# ───────────────────────────────────────────────────────────────
# RIEPILOGO DEGLI ERRORI TROVATI
# ───────────────────────────────────────────────────────────────
fp_indices, fn_indices = trova_errori(y_test_binary, y_pred)

total_fp_indices = len(fp_indices)
total_fn_indices = len(fn_indices)
total_error = total_fp_indices + total_fn_indices
error_percentage = (total_error) / len(y_test_binary) * 100

print("=" * 60)
print("ANALISI DEGLI ERRORI DI CLASSIFICAZIONE")
print("=" * 60)
print(f"\nDistribuzione degli errori:")
print(f"   • False Positive (FP): {total_fp_indices} immagini → animali scambiati per veicoli")
print(f"   • False Negative (FN): {total_fn_indices} immagini → veicoli scambiati per animali")
print(f"\n   Totale errori: {total_error} su {len(y_test_binary)} immagini "
      f"({error_percentage:.2f}%)")


In [ ]:
# ───────────────────────────────────────────────────────────────
# VISUALIZZAZIONE DEGLI ERRORI DI CLASSIFICAZIONE
# ───────────────────────────────────────────────────────────────
# Scopo: osservare esempi concreti di immagini classificate in modo errato
#        per comprendere i pattern di errore del modello.
#        - False Positive (FP): animali scambiati per veicoli
#        - False Negative (FN): veicoli scambiati per animali
#        Questa analisi qualitativa aiuta a capire *dove* la rete sbaglia.
# ───────────────────────────────────────────────────────────────

def visualizza_errori(x, y_true, y_pred, y_pred_probs, error_indices, 
                      error_type, n_samples=10):
    """
    Visualizza un campione di immagini classificate erroneamente.

    Parametri:
        x (ndarray): immagini originali (non normalizzate)
        y_true (ndarray): etichette reali binarie (0=animale, 1=veicolo)
        y_pred (ndarray): etichette predette binarie
        y_pred_probs (ndarray): probabilità predette (sigmoid output)
        error_indices (ndarray): indici dei campioni errati
        error_type (str): tipo di errore ('FP' o 'FN')
        n_samples (int): numero massimo di esempi da mostrare
    """

    # ───────────────────────────────────────────────
    # 1. Controllo preliminare
    # ───────────────────────────────────────────────
    if len(error_indices) == 0:
        print(f"Nessun errore di tipo {error_type} trovato!")
        return

    # Limitiamo il numero di esempi visualizzati
    n_to_show = min(n_samples, len(error_indices))

    # Selezione casuale di alcuni indici di errore
    selected_indices = np.random.choice(error_indices, n_to_show, replace=False)

    # ───────────────────────────────────────────────
    # 2. Impostazione della griglia di visualizzazione
    # ───────────────────────────────────────────────
    rows, cols = 2, 5
    fig, axes = plt.subplots(rows, cols, figsize=(16, 7))

    # Titolo generale in base al tipo di errore
    tipo_errore = ("False Positive (Animali → Veicoli)" 
                   if error_type == 'FP' 
                   else "False Negative (Veicoli → Animali)")
    fig.suptitle(f"Esempi di {tipo_errore}", fontsize=16, fontweight='bold')

    # ───────────────────────────────────────────────
    # 3. Visualizzazione delle immagini errate
    # ───────────────────────────────────────────────
    for idx, ax in enumerate(axes.flat):
        if idx < n_to_show:
            img_idx = selected_indices[idx]

            # Recupero immagine e dati associati
            img = x[img_idx]
            true_label = y_true[img_idx][0]
            pred_label = y_pred[img_idx][0]
            prob = y_pred_probs[img_idx][0]

            # Recupero della classe originale CIFAR-10
            original_class = class_names[y_test[img_idx][0]]

            # Visualizzazione immagine
            ax.imshow(img)

            # Costruzione titolo informativo
            vera_str = "Animale" if true_label == 0 else "Veicolo"
            pred_str = "Animale" if pred_label == 0 else "Veicolo"

            title = f"{original_class}\n"
            title += f"Vero: {vera_str}\n"
            title += f"Pred: {pred_str} ({prob:.2f})"

            # Titolo in rosso per evidenziare l'errore
            ax.set_title(title, fontsize=9, color='red')
            ax.axis('off')
        else:
            ax.axis('off')

    plt.tight_layout()
    plt.show()


# ───────────────────────────────────────────────────────────────
# VISUALIZZAZIONE DEI DUE TIPI DI ERRORI
# ───────────────────────────────────────────────────────────────

# 1. False Positive → animali scambiati per veicoli
print("\n" + "="*60)
print("FALSE POSITIVE → Animali scambiati per Veicoli")
print("="*60)
visualizza_errori(x_test, y_test_binary, y_pred, y_pred_probs, 
                  fp_indices, 'FP', n_samples=10)

# 2. False Negative → veicoli scambiati per animali
print("\n" + "="*60)
print("FALSE NEGATIVE → Veicoli scambiati per Animali")
print("="*60)
visualizza_errori(x_test, y_test_binary, y_pred, y_pred_probs, 
                  fn_indices, 'FN', n_samples=10)


### Riflessioni sugli errori

#### Possibili cause delle misclassificazioni:

1. **Bassa risoluzione**: le immagini CIFAR-10 sono solo 32x32 pixel, rendendo difficile distinguere dettagli fini

2. **Ambiguità visiva**: alcune immagini possono essere sfocate, con angolazioni inusuali o parzialmente occluse

3. **Somiglianza tra classi**: 
   - uccelli in volo potrebbero sembrare aerei (forma simile, sfondo cielo)
   - animali con forme regolari potrebbero sembrare oggetti meccanici

4. **Limitazioni del dataset**: CIFAR-10 non è stato progettato specificamente per questa distinzione binaria

#### Miglioramenti possibili del modello:

- **Data Augmentation**: rotazioni, flip, zoom per aumentare la varietà
- **Architetture più profonde**: più layer convoluzionali per catturare pattern complessi
- **Transfer Learning**: usare reti pre-addestrate (es. VGG, ResNet) come base
- **Dataset più ricco**: immagini ad alta risoluzione e più bilanciate
- **Regolarizzazione avanzata**: tecniche come L2 regularization o dropout più sofisticato


---

## 7. Opzionale: esempi di predizioni corrette

Per completezza, mostro anche alcuni esempi di immagini classificate **correttamente** dal mio modello.


In [ ]:
# ───────────────────────────────────────────────────────────────
# VISUALIZZAZIONE DELLE CLASSIFICAZIONI CORRETTE
# ───────────────────────────────────────────────────────────────
# Scopo: mostrare esempi di immagini classificate correttamente
#        dal modello (sia animali che veicoli), per confrontarli
#        con i casi errati e verificare i pattern di successo.
#        Permette di capire su quali tipi di immagini la rete
#        si comporta in modo affidabile.
# ───────────────────────────────────────────────────────────────

def visualizza_predizioni_corrette(x, y_true, y_pred, y_pred_probs, n_samples=20):
    """
    Visualizza esempi di immagini classificate correttamente.

    Parametri:
        x (ndarray): immagini originali (non normalizzate)
        y_true (ndarray): etichette reali binarie (0=animale, 1=veicolo)
        y_pred (ndarray): etichette predette binarie
        y_pred_probs (ndarray): probabilità predette (output sigmoide)
        n_samples (int): numero massimo di esempi da mostrare
    """

    # ───────────────────────────────────────────────
    # 1. Identificazione delle predizioni corrette
    # ───────────────────────────────────────────────
    correct_indices = np.where(y_pred.flatten() == y_true.flatten())[0]

    # ───────────────────────────────────────────────
    # 2. Selezione casuale di un sottoinsieme
    # ───────────────────────────────────────────────
    n_to_show = min(n_samples, len(correct_indices))
    selected = np.random.choice(correct_indices, n_to_show, replace=False)

    # ───────────────────────────────────────────────
    # 3. Impostazione della griglia di visualizzazione
    # ───────────────────────────────────────────────
    fig, axes = plt.subplots(4, 5, figsize=(16, 13))
    fig.suptitle('Esempi di Classificazioni CORRETTE', 
                 fontsize=16, fontweight='bold', color='green')

    # ───────────────────────────────────────────────
    # 4. Visualizzazione delle immagini corrette
    # ───────────────────────────────────────────────
    for idx, ax in enumerate(axes.flat):
        if idx < len(selected):
            img_idx = selected[idx]
            img = x[img_idx]
            true_label = y_true[img_idx][0]
            prob = y_pred_probs[img_idx][0]

            # Recupero della classe originale CIFAR-10
            original_class = class_names[y_test[img_idx][0]]

            # Mostra immagine
            ax.imshow(img)

            # Costruzione titolo descrittivo
            label_str = "Veicolo" if true_label == 1 else "Animale"
            conf = prob if true_label == 1 else 1 - prob
            title = f"{original_class}\n {label_str}\nConfidenza: {conf:.2f}"

            # Titolo in verde per enfatizzare la correttezza
            ax.set_title(title, fontsize=9, color='green', fontweight='bold')
            ax.axis('off')
        else:
            ax.axis('off')

    plt.tight_layout()
    plt.show()

    # ───────────────────────────────────────────────
    # 5. Riepilogo numerico dei risultati
    # ───────────────────────────────────────────────
    correct_ratio = len(correct_indices) / len(y_true)
    print(f"\nIl modello ha classificato correttamente "
          f"{len(correct_indices)} immagini su {len(y_true)} "
          f"({correct_ratio * 100:.2f}%)")


# ───────────────────────────────────────────────────────────────
# ESEMPIO DI VISUALIZZAZIONE
# ───────────────────────────────────────────────────────────────
visualizza_predizioni_corrette(x_test, y_test_binary, y_pred, y_pred_probs, n_samples=20)
